# Day 8: Seven Segment Search

You barely reach the safety of the cave when the whale smashes into the cave mouth, collapsing it. Sensors indicate another exit to this cave at a much greater depth, so you have no choice but to press on.

As your submarine slowly makes its way through the cave system, you notice that the four-digit seven-segment displays in your submarine are malfunctioning; they must have been damaged during the escape. You'll be in a lot of trouble without them, so you'd better figure out what's wrong.

Each digit of a seven-segment display is rendered by turning on or off any of seven segments named a through g:

So, to render a 1, only segments c and f would be turned on; the rest would be off. To render a 7, only segments a, c, and f would be turned on.

The problem is that the signals which control the segments have been mixed up on each display. The submarine is still trying to display numbers by producing output on signal wires a through g, but those wires are connected to segments randomly. Worse, the wire/segment connections are mixed up separately for each four-digit display! (All of the digits within a display use the same connections, though.)

For each display, you watch the changing signals for a while, make a note of all ten unique signal patterns you see, and then write down a single four digit output value (your puzzle input). Using the signal patterns, you should be able to work out which pattern corresponds to which digit.

Each entry consists of ten unique signal patterns, a | delimiter, and finally the four digit output value. Within an entry, the same wire/segment connections are used (but you don't know what the connections actually are). The unique signal patterns correspond to the ten different ways the submarine tries to render a digit using the current wire/segment connections. Because 7 is the only digit that uses three segments, dab in the above example means that to render a 7, signal lines d, a, and b are on. Because 4 is the only digit that uses four segments, eafb means that to render a 4, signal lines e, a, f, and b are on.

Using this information, you should be able to work out which combination of signal wires corresponds to each of the ten digits. Then, you can decode the four digit output value. Unfortunately, in the above example, all of the digits in the output value (cdfeb fcadb cdfeb cdbaf) use five segments and are more difficult to deduce.

In [ ]:
[Flags]
enum Signal { a = 1 , b = 2, c = 4, d = 8, e = 16, f = 32, g = 64 };

static (int,Signal) ParseSignal(string s)
    => (s.Length, Enum.Parse<Signal>(string.Join(',', s.ToCharArray())));
    
var digits = "abcefg cf acdeg acdfg bcdf abdfg abdefg acf abcdefg abcdfg"
    .Split(' ').Select(ParseSignal).ToArray();
digits


index,Item1,Item2
0,6,"a, b, c, e, f, g"
1,2,"c, f"
2,5,"a, c, d, e, g"
3,5,"a, c, d, f, g"
4,4,"b, c, d, f"
5,5,"a, b, d, f, g"
6,6,"a, b, d, e, f, g"
7,3,"a, c, f"
8,7,"a, b, c, d, e, f, g"
9,6,"a, b, c, d, f, g"


In [ ]:
record Note((int,Signal)[] Unique, (int,Signal)[] Output);
    
static Note ParseNote(string s) {
    var parts = s.Split('|', StringSplitOptions.TrimEntries)
        .Select(s => s.Split(' ').Select(ParseSignal).ToArray()
        ).ToArray();
    return new Note(parts[0], parts[1]);
}

In [ ]:
using System.IO;
// var input = File.ReadAllLines(@"day-08.sample");
var input = File.ReadAllLines(@"day-08.input");

var note = ParseNote("acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf");

var notes = input.Select(ParseNote).ToArray();
notes.Length

200

## Part 1

For now, focus on the easy digits.

Because the digits 1, 4, 7, and 8 each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Counting only digits in the output values (the part after | on each line), in the above example, there are 26 instances of digits that use a unique number of segments (highlighted above).

In [ ]:
static bool IsEasyDigit((int,Signal) pattern)
    => pattern.Item1 switch {
        2 or 4 or 3 or 7 => true,
        _ => false
    };

display(note.Output.Count(p => IsEasyDigit(p)));

notes.SelectMany(n => n.Output).Count(p => IsEasyDigit(p))

0

310

## Part 2

Through a little deduction, you should now be able to determine the remaining digits.

For each entry, determine all of the wire/segment connections and decode the four-digit output values. What do you get if you add up all of the output values?

In [ ]:
static Signal[] Decode((int,Signal)[] unique) {
    var result = new Signal[10];
    var possible = unique.ToList();

    Signal Take1(int len, Func<Signal, bool> check) {
        var result = possible.Single(u => u.Item1 == len && check(u.Item2));
        possible.Remove(result);
        return result.Item2;
    }

    result[1] = Take1(2, u => true);
    result[4] = Take1(4, u => true);
    result[7] = Take1(3, u => true);
    result[8] = Take1(7, u => true);

    result[3] = Take1(5, u => (u & result[1]) == result[1]);
    result[9] = Take1(6, u => (u & result[3]) == result[3]);    

    result[5] = Take1(5, u => (u & result[9]) == u);
    result[6] = Take1(6, u => (u & result[1]) != result[1]);

    result[0] = Take1(6, u => true);
    result[2] = Take1(5, u => true);

    // display(possible);

    return result;
}

int Value(Note note) {
    var decode = Decode(note.Unique).Select((s, i) => (s, i)).ToDictionary(t => t.Item1, t => t.Item2);

    return note.Output
        .Select(s => decode[s.Item2]).Aggregate(0, (a, v) => a * 10 + v);
}

display(Value(note));

notes.Select(Value).Sum()

5353

915941